In [ ]:
import numpy as np
import pandas as pd
import regex as re
import random as rn
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,average_precision_score, precision_score,precision_recall_curve
from tqdm.notebook import tqdm
from tqdm import trange
import warnings
import pickle
import nltk
import math
import os
import json
import random
import re
import torch
import torch.nn as nn
import spacy 
from spacy import displacy
#import neptune.new as neptune                          
from torch.utils.data import (DataLoader, RandomSampler, WeightedRandomSampler, SequentialSampler, TensorDataset, Dataset)

In [ ]:
!pip install emoji -q
!pip install datasets -q
!pip install rouge_score -q


!pip install transformers -q
!pip install sentencepiece -q
!pip install pandarallel -q

from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel

from transformers import AdamW, get_linear_schedule_with_warmup,PegasusConfig, PegasusTokenizer, PegasusForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, set_seed, TrainingArguments, Trainer, default_data_collator, AdamW, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments


     |████████████████████████████████| 170 kB 5.0 MB/s 
     |████████████████████████████████| 298 kB 5.2 MB/s 
     |████████████████████████████████| 61 kB 550 kB/s 
     |████████████████████████████████| 132 kB 5.7 MB/s 
     |████████████████████████████████| 1.1 MB 56.0 MB/s 
     |████████████████████████████████| 243 kB 70.5 MB/s 
     |████████████████████████████████| 271 kB 57.9 MB/s 
     |████████████████████████████████| 192 kB 70.3 MB/s 
     |████████████████████████████████| 160 kB 60.4 MB/s 
     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 62.3 MB/s 
     |████████████████████████████████| 895 kB 77.2 MB/s 
     |████████████████████████████████| 3.3 MB 54.6 MB/s 
     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
model_name = 'google/pegasus-xsum'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = PegasusConfig.from_pretrained(model_name)
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Mounted at /content/drive


In [ ]:
reviews_path = '/content/drive/MyDrive/gen_datasets/res_reviews_final.json'
# reviews_path = '/content/drive/MyDrive/restaurant_reviews/res_reviews_final.json'
import json
f = open(reviews_path)
data = json.load(f)
data = json.loads(data)

In [ ]:
x = {}
for key_ in data.keys():
  for a in data[key_]['review_list']:
    if(a['rating']!=None):
      x[a['text']] = a['rating']

In [ ]:
sorted_vals = sorted(x.items(), key=lambda item: item[1])

In [ ]:
reviews = []
for j in sorted_vals:
  reviews.append(j[0])

In [ ]:
num_kp = int(len(reviews)/50)
input = []
for i in range(len(reviews)):
  if(i%num_kp==0):
    input.append(reviews[i])
    src_text = ''.join(input)
    batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    print(i,tgt_text)
    input = []

0 ["I've been to this location before as well as other and used to love it and arrived at the airport early just to eat here. I've been to this location before as well as other and used to love it and arrived at the airport early just to eat here."]
28 ["It's a shame that Brookline Hospital is charging $9 for parking."]
56 ["I've been to Boston Beerworks before."]
84 ["This was the worst meal I've ever had."]
112 ['This is one of the worst restaurants I have ever been to.']
140 ["This is the worst restaurant we've ever been to."]
168 ["McDonald's in the food court area of terminal B is a disappointment."]
196 ["I've eaten at this location and others at the past and this is the worst experience I've had so far."]
224 ['The lobster roll was terrible!']
252 ["I've been to this place a few times and it's always good!"]
280 ['The food court at the Royal Victoria Hospital is one of the best places to eat in London.']
308 ["It's lobster roll time."]
336 ["I'm not a huge fan of airport food, b